In [2]:
import torch
print(torch.backends.mps.is_available())

True


In [16]:
from langchain_community.llms import Ollama
llm = Ollama(model="mistral")

llm.invoke("Tell me another joke")

" Why don't scientists trust atoms?\n\nBecause they make up everything!\n\nOr, if you prefer a classic:\n\nWhy did the chicken cross the playground?\n\nTo get to the other slide!"

In [11]:
# import torch
# from transformers import AutoModelForCausalLM, AutoTokenizer

# model_id = "mistralai/Mixtral-8x7B-v0.1"
# tokenizer = AutoTokenizer.from_pretrained(model_id)

# model = AutoModelForCausalLM.from_pretrained(model_id, load_in_4bit=True, device_map='mps')

# text = "Hello my name is"
# inputs = tokenizer(text, return_tensors="pt").to("mps")

# outputs = model.generate(**inputs, max_new_tokens=20)
# print(tokenizer.decode(outputs[0], skip_special_tokens=True))

In [17]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

In [27]:
# base_model_id = "mistralai/Mistral-7B-v0.1"
# bnb_config = BitsAndBytesConfig(
#     load_in_4bit = True,
#     bnb_4bit_use_double_quant = True,
#     bnb_4bit_quant_type="nf4",
#     bnb_4bit_compute_dtype=torch.bfloat16
# )
# model = AutoModelForCausalLM.from_pretrained(base_model_id, quantization_config=bnb_config, device_map='mps')

In [25]:
torch.backends.mps.is_available()

True

In [22]:
client.close()

In [30]:
from pathlib import Path
import qdrant_client
from llama_index import VectorStoreIndex, ServiceContext, SimpleDirectoryReader
from llama_index.llms import Ollama
from llama_index.storage.storage_context import StorageContext
from llama_index.vector_stores.qdrant import QdrantVectorStore

# Loading the documents from the disk
documents = SimpleDirectoryReader("./data").load_data()

# Initializing the vector store with Qdrant
client = qdrant_client.QdrantClient(path="./qdrant_data")
vector_store = QdrantVectorStore(client=client, collection_name="springboot")
storage_context = StorageContext.from_defaults(vector_store=vector_store)

# Initializing the Large Language Model (LLM) with Ollama
# The request_timeout may need to be adjusted depending on the system's performance capabilities
llm = Ollama(model="mistral", request_timeout=120.0)
service_context = ServiceContext.from_defaults(llm=llm, embed_model="local")

# Creating the index, which includes embedding the documents into the vector store
index = VectorStoreIndex.from_documents(documents, service_context=service_context, storage_context=storage_context)

# Querying the index with a specific question
query_engine = index.as_query_engine()
prompt = (
  "What are the optimization methods, show me only the method name"
)
response = query_engine.query(prompt)
client.close()
print(response)

 I cannot directly provide a list of optimization methods from the context information as it does not mention any specific optimization methods aside from early stopping which is actually a regularization technique rather than an optimization method itself.


In [29]:
# Just runs .complete to make sure the LLM is listening
from llama_index.llms import Ollama

llm = Ollama(model="mistral")

response = llm.complete("What is the history of LEGO?")
print(response)

 LEGO is a Danish toy manufacturing company that produces interlocking brick system toys. The company was founded on August 10, 1932, by Ole Kirk Christiansen in Billund, Denmark. Originally named "Leg Godt," which means "Play Well" in Danish, the company started producing wooden toys.

In 1947, Christiansen acquired a small factory that was producing plastic toys, and he began experimenting with plastic injection molding. In 1949, he produced the first LEGO bricks, which were made from a strong, durable plastic called acrylonitrile butadiene styrene (ABS). The bricks were designed to interlock, allowing for easy construction of various structures and models.

The first LEGO sets were marketed towards children and were sold in Europe. In the 1950s, Christiansen started producing more complex sets that included figures and accessories. In the late 1950s, the company introduced the iconic LEGO minifigure, which has remained a staple of LEGO sets to this day.

Throughout the 1960s and 197

In [24]:
from sqlalchemy import (
    create_engine,
    MetaData,
    Table,
    Column,
    String,
    Integer,
    select,
)



In [25]:
engine = create_engine("sqlite:///:memory:")
metadata_obj = MetaData()

In [26]:
# create city SQL table
table_name = "city_stats"
city_stats_table = Table(
    table_name,
    metadata_obj,
    Column("city_name", String(16), primary_key=True),
    Column("population", Integer),
    Column("country", String(16), nullable=False),
)
metadata_obj.create_all(engine)

In [32]:
from llama_index import SQLDatabase, ServiceContext
sql_database = SQLDatabase(engine, include_tables=["city_stats"])
from sqlalchemy import insert

rows = [
    {"city_name": "Toronto", "population": 2930000, "country": "Canada"},
    {"city_name": "Tokyo", "population": 13960000, "country": "Japan"},
    {
        "city_name": "Chicago",
        "population": 2679000,
        "country": "United States",
    },
    {"city_name": "Seoul", "population": 9776000, "country": "South Korea"},
]
for row in rows:
    stmt = insert(city_stats_table).values(**row)
    with engine.begin() as connection:
        cursor = connection.execute(stmt)

In [34]:
llm = Ollama(model="mistral")
service_context = ServiceContext.from_defaults(llm=llm,  embed_model="local")
sql_database = SQLDatabase(engine, include_tables=["city_stats"])

In [35]:
stmt = select(
    city_stats_table.c.city_name,
    city_stats_table.c.population,
    city_stats_table.c.country,
).select_from(city_stats_table)

with engine.connect() as connection:
    results = connection.execute(stmt).fetchall()
    print(results)

[('Toronto', 2930000, 'Canada'), ('Tokyo', 13960000, 'Japan'), ('Chicago', 2679000, 'United States'), ('Seoul', 9776000, 'South Korea')]


In [36]:
from sqlalchemy import text

with engine.connect() as con:
    rows = con.execute(text("SELECT city_name from city_stats"))
    for row in rows:
        print(row)

('Chicago',)
('Seoul',)
('Tokyo',)
('Toronto',)


In [39]:
from llama_index.indices.struct_store.sql_query import NLSQLTableQueryEngine

query_engine = NLSQLTableQueryEngine(
    sql_database=sql_database,
    tables=["city_stats"],
    service_context = service_context
)
query_str = "Which city has the highest population?"
response = query_engine.query(query_str)

In [40]:
response

Response(response=' Based on the query results, Tokyo is the city with the highest population. Therefore, the response to the question "Which city has the highest population?" would be:\n\n"The city with the highest population is Tokyo, with a population of approximately 13,960,000 people."', source_nodes=[NodeWithScore(node=TextNode(id_='1faf40ab-8691-4f0d-87e6-072049b3689e', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text="[('Tokyo', 13960000)]", start_char_idx=None, end_char_idx=None, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n'), score=None)], metadata={'1faf40ab-8691-4f0d-87e6-072049b3689e': {}, 'sql_query': 'SELECT city_name, population FROM city_stats ORDER BY population DESC LIMIT 1;', 'result': [('Tokyo', 13960000)], 'col_keys': ['city_name', 'population']})

In [72]:
from llama_index.indices.struct_store.sql_query import (
    SQLTableRetrieverQueryEngine,
)
from IPython.display import Markdown, display
from llama_index.objects import (
    SQLTableNodeMapping,
    ObjectIndex,
    SQLTableSchema,
)
from llama_index import VectorStoreIndex

# set Logging to DEBUG for more detailed outputs
table_node_mapping = SQLTableNodeMapping(sql_database)
table_schema_objs = [
    (SQLTableSchema(table_name="city_stats"))
]  # add a SQLTableSchema for each table

obj_index = ObjectIndex.from_objects(
    table_schema_objs,
    table_node_mapping,
    VectorStoreIndex,
    service_context = service_context
)
query_engine = SQLTableRetrieverQueryEngine(
    sql_database, obj_index.as_retriever(similarity_top_k=5),
    service_context = service_context
)

response = query_engine.query("Which city has the highest population?")
display(Markdown(f"<b>{response}</b>"))

<b> Based on the query results, Tokyo is the city with the highest population, with a population of approximately 13,960,000 people.</b>

In [71]:
response.metadata['sql_query']






d

'SELECT city_name, population FROM city_stats ORDER BY population DESC LIMIT 1;'

In [50]:
from llama_index.retrievers import NLSQLRetriever

# default retrieval (return_raw=True)
nl_sql_retriever = NLSQLRetriever(
    sql_database, tables=["city_stats"], return_raw=True,
    service_context = service_context  
)

results = nl_sql_retriever.retrieve(
    "Return the top 5 cities (along with their populations) with the highest population."
)
from llama_index.response.notebook_utils import display_source_node
for n in results:
    display_source_node(n, show_source_metadata=True)

**Node ID:** 17db383a-f008-4e1e-9f19-f7fda59a2c67<br>**Similarity:** None<br>**Text:** [('Tokyo', 13960000), ('Seoul', 9776000), ('Toronto', 2930000), ('Chicago', 2679000)]<br>**Metadata:** {}<br>

In [52]:
from llama_index.query_engine import RetrieverQueryEngine

query_engine = RetrieverQueryEngine.from_args(nl_sql_retriever, service_context=service_context)
response = query_engine.query(
    "Return the top 5 cities (along with their populations) with the highest population."
)

In [53]:
print(str(response))

 To retrieve the top 5 cities with the highest population, you can use a SQL query in the following format:
```sql
SELECT city_name, population 
FROM city_stats 
ORDER BY population DESC 
LIMIT 5;
```
However, based on the context provided, it seems that there is an error in the syntax of the query. To correct this error, make sure to include a semicolon at the end of the statement:
```sql
SELECT city_name, population 
FROM city_stats 
ORDER BY population DESC 
LIMIT 5;
```
This corrected query will return the top 5 cities with the highest population from the `city_stats` table.
